In [18]:
from small_llm_reasoning.evaluation.gsm8k import questions, answers, get_score
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig
import os
from datasets import load_from_disk
from vllm import LLM, SamplingParams
import pandas as pd
import json

cache_dir = '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/'
os.environ['TRANSFORMERS_CACHE'] = cache_dir


In [3]:
data_path= "../datasets/gsm8k"
data= load_from_disk(f"{data_path}/tokenized/LLaMA3B/pretrained/test/eight-shot/")


In [12]:
# Loading tokenizer
hf_token = os.getenv("hf_token")

model_name= "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, cache_dir=cache_dir)
tokenizer.pad_token_id = tokenizer.eos_token_id



In [19]:
sampling_params = SamplingParams(n=1,
                                 temperature=0,
                                 max_tokens=500,
                                 logprobs=2
                                )

model = LLM(
        model=model_name, 
        tensor_parallel_size=1) 



WARNING 03-05 21:53:08 arg_utils.py:862] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 03-05 21:53:08 config.py:999] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 03-05 21:53:08 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='meta-llama/Llama-3.2-3B', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.58it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  2.49it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:00<00:00,  2.29it/s]


INFO 03-05 21:53:10 model_runner.py:926] Loading model weights took 5.9847 GB


INFO 03-05 21:53:11 gpu_executor.py:122] # GPU blocks: 19194, # CPU blocks: 2340
INFO 03-05 21:53:11 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-05 21:53:11 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-05 21:53:22 model_runner.py:1335] Graph capturing finished in 11 secs.


In [24]:
len(data['input_ids'][0]['prompt_token_ids'])

1064

In [ ]:
outputs = model.generate([data['input_ids'][0]], sampling_params)

Processed prompts:  28%|██▊       | 337/1195 [01:05<02:40,  5.36it/s, est. speed input: 5486.67 toks/s, output: 2589.19 toks/s]

In [ ]:
outputs